In [ ]:
import geopandas as gpd
import pandas as pd

# Caminho do shapefile
caminho = "data/PRODES_AMZ_Municipal_2023.shp"

# Ler o shapefile
gdf = gpd.read_file(caminho)

print("Colunas encontradas no shapefile:")
print(gdf.columns)


# Agrupar por estado e ano, somando área total desmatada
# Salvar como CSV

gdf.to_csv('data/desmatamento_por_estado.csv', index=False)

print("✅ Arquivo CSV agregado salvo com sucesso em data/desmatamento_por_estado.csv")



In [ ]:
import pandas as pd

url = "https://apisidra.ibge.gov.br/values/t/5938/n6/all/v/37/p/2021"
df = pd.read_json(url)
df.to_csv("data/pib_municipal_2023.csv", index=False)

print("✅ PIB municipal 2021 salvo com sucesso!")


In [ ]:
import pandas as pd
import os

# Caminho da base original
caminho = "data/desmatamento_por_estado.csv"

# Criar pasta de saída
os.makedirs("data/limpos", exist_ok=True)

# 1️⃣ Ler a base
df = pd.read_csv(caminho)
print("✅ Base carregada com sucesso!")
print("📊 Linhas:", len(df))

# 2️⃣ Manter apenas colunas importantes
colunas_importantes = [
    "fid",
    "state",
    "image_date",
    "area_km"
]
df = df[colunas_importantes]
print("🔹 Colunas mantidas:", list(df.columns))

# 3️⃣ Corrigir tipos de dados
df["area_km"] = pd.to_numeric(df["area_km"], errors="coerce")
df["image_date"] = pd.to_datetime(df["image_date"], errors="coerce")

# 4️⃣ Filtrar apenas registros de 2021 (sem criar coluna year permanente)
df = df.loc[df["image_date"].dt.year == 2021]

# 5️⃣ Remover linhas com dados inválidos
df = df.dropna(subset=["area_km", "image_date"])

# 6️⃣ Padronizar nomes das colunas
df.columns = df.columns.str.lower()

# 7️⃣ Salvar versão limpa (sem coluna year)
output = "data/limpos/desmatamento_2021.csv"
df.to_csv(output, index=False, encoding="utf-8-sig")

print("✅ Base filtrada por 2021 salva com sucesso!")
print(df.head())


ModuleNotFoundError: No module named 'pandas'